<a href="https://colab.research.google.com/github/dbrody112/Frequentist-Machine-Learning/blob/main/Dan_Brody_Model_Assessment_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model Assessment (Incorrect Way)**



In [ ]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from collections import Counter
from IPython.core.display import display, HTML
sns.set_style('darkgrid')
from sklearn import datasets, svm
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import plotly.express as px
from sklearn.metrics import accuracy_score

#CREATING BINARY CLASSIFICATION GAUSSIAN DATASET

X,y = sklearn.datasets.make_classification(n_samples = 50, n_features = 5000, weights = [0.50], random_state = 25)
df = pd.DataFrame(X)
df["target"] = y

#GETTING BEST 100 COLUMNS CORRELATED TO TARGET

z = (df.corr()['target']).drop('target').values
z.sort()
best_hundred = []
for i in range(100):
    best_hundred.append(z[len(z)-1-i])
a = (df.corr()['target']).drop('target')[(df.corr()['target']).drop('target') >= min(best_hundred)].index
new_x = []

#CREATING DATASET OF BEST 100 COLUMNS

for i in a:
    new_x.append(df.values[:,i])
new_x = np.array(new_x)
new_x_transpose = np.transpose(new_x)

#DECLARING 1-NEIGHBOR KNEIGHBORS

neigh = KNeighborsClassifier(n_neighbors = 1)

np.random.seed(25)
kf = KFold(n_splits = 5,shuffle = True)

#USING K-FOLD TO ITERATE THROUGH 50 SPLITS OF DATASET

accuracy_model=[]
for train_index, test_index in kf.split(new_x_transpose):

    # Split train-test

    X_train, X_test = new_x_transpose[train_index], new_x_transpose[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model

    model = neigh.fit(X_train, y_train)
    pred = model.predict(X_test)
    
        
    # Append to accuracy_model the accuracy of the model

    accuracy_model.append(accuracy_score(y_test,pred))

print(f"Incorrect Model Assessment Accuracy is {np.average(accuracy_model)*100}% while benchmark with zeros is {accuracy_score(y_test,np.zeros((len(y_test),1)))*100}% and benchmark with ones is {accuracy_score(y_test,np.ones((len(y_test),1)))*100}%")

Incorrect Model Assessment Accuracy is 94.0% while benchmark with zeros is 40.0% and benchmark with ones is 60.0%


# **Model Assessment (The Correct Way)**

In [ ]:
#SPLITTING DATASET TO ISOLATE ONE SAMPLE
s = 1
kf2 = KFold(n_splits = 5,shuffle = True)
accuracy_model = []
for train_index, test_index in kf2.split(X):
    # Split train-test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model

    #REMOVING SAMPLE SELECTED FROM KFOLD
    
    test_remove = []
    for i in test_index:
      test_remove.append(i)
    
    df = pd.DataFrame(X)
    df["target"] = y
    df.drop(test_remove,inplace = True)
    
    #FINDING 100 MOST CORRELATED COLUMNS

    z_correct = (df.corr()['target']).drop('target').values
    z_correct.sort()
    best_hundred_correct = []

    for i in range(100):
      best_hundred_correct.append(z_correct[len(z_correct)-1-i])
    a_correct = (df.corr()['target']).drop('target')[(df.corr()['target']).drop('target') >= min(best_hundred)].index

    new_x_correct = []
    for i in a_correct:
      new_x_correct.append(df.values[:,i])
    new_x_correct_transpose = np.transpose(new_x_correct)

    new_x_test_correct = []
    for i in a_correct:
      new_x_test_correct.append(X_test[:,i])
    new_x_test_correct_transpose = np.transpose(new_x_test_correct)

      #IMPLEMENTING KNEIGHBORS 

    neigh = KNeighborsClassifier(n_neighbors = 1)
    neigh.fit(new_x_correct_transpose,df['target'])
    pred = neigh.predict(new_x_test_correct_transpose)

    accuracy_model.append([accuracy_score((y_test),pred),a_correct])
    
    print(f"Kfold # {s} is completed. Waiting on next fold...")
    print("\n")
    s+=1
    


Kfold # 1 is completed. Waiting on next fold...


Kfold # 2 is completed. Waiting on next fold...


Kfold # 3 is completed. Waiting on next fold...


Kfold # 4 is completed. Waiting on next fold...


Kfold # 5 is completed. Waiting on next fold...




In [ ]:
accuracy_array = []
for i in range(5):
  accuracy_array.append(accuracy_model[i][0])
  print(accuracy_model[i][0])

0.5
0.4
0.5
0.5
0.5


In [ ]:
#error before was saying "for i in a " rather than " for i in a_correct"

print(f"Correct Model Assessment Accuracy is {np.average(accuracy_array)*100}%  while benchmark with zeros is {accuracy_score(y_test,np.zeros((len(y_test),1)))*100}% and benchmark with ones is {accuracy_score(y_test,np.ones((len(y_test),1)))*100}%")

Correct Model Assessment Accuracy is 48.0%  while benchmark with zeros is 50.0% and benchmark with ones is 50.0%


In [ ]:
#SPLITTING DATASET TO ISOLATE TEN SAMPLES

kf2 = KFold(n_splits = 5, shuffle = True)
for train_index, test_index in kf2.split(X):
    # Split train-test
    new_X_train, new_X_test = X[train_index], X[test_index]
    new_y_train, new_y_test = y[train_index], y[test_index]
    # Train the model

In [ ]:
#CREATING DATAFRAME USING CORRECT METHOD

a_correct = accuracy_model[np.random.randint(0,5)][1]
df2 = pd.DataFrame(new_X_test)
df2_correct = []
for i in a_correct:
  df2_correct.append(df2.values[:,i])

In [ ]:
df2_correct = np.array(df2_correct)
df2_correct_transpose = np.transpose(df2_correct)
df2_correct_transpose
df3 = pd.DataFrame(df2_correct_transpose)

In [ ]:
#PLOTTING DISTRIBUTION OF CORRELATIONS

df3["target"] = new_y_test
fig = px.histogram(df3, x=df3.corr()["target"].drop('target'), title = "Correct Model Assessment")
f

In [ ]:
#CREATING DATAFRAME USING CORRECT METHOD

df2_incorrect = []
for i in a:
  df2_incorrect.append(df2.values[:,i])
df2_incorrect = np.array(df2_incorrect)
df2_incorrect_transpose = np.transpose(df2_incorrect)
df2_incorrect_transpose
df4 = pd.DataFrame(df2_incorrect_transpose)
df4["target"] = new_y_test

#PLOTTING DISTRIBUTION OF CORRELATIONS

fig = px.histogram(df4, x=df4.corr()["target"].drop('target'), title = "Incorrect Model Assessment")
fig.show()